In [2]:
### Read PDF

# from dotenv import load_dotenv
# Charger les variables d'environnement
# load_dotenv()
# Faire lettre motivation automatique
from datetime import datetime
import locale

import re

In [3]:
from PyPDF2 import PdfReader

def read_pdf(file_path: str) -> str:
    """Lit et extrait le texte d'un CV au format PDF d'une seule page.

    Args:
        file_path (str): Chemin d'accès vers le fichier PDF à lire

    Returns:
        str: Texte brut extrait du PDF

    Raises:
        FileNotFoundError: Si le fichier PDF n'existe pas
        IndexError: Si le PDF est vide
        Exception: Pour toute autre erreur lors de la lecture du PDF
    """
    try:
        reader = PdfReader(file_path)
        if len(reader.pages) == 0:
            raise IndexError("Le PDF est vide")
        
        page = reader.pages[0]  # Lecture de la première page uniquement
        text_brut = page.extract_text()
        
        if not text_brut:
            raise Exception("Aucun texte n'a pu être extrait du PDF")
            
        return text_brut

    except FileNotFoundError:
        raise FileNotFoundError(f"Le fichier {file_path} n'existe pas")
    except IndexError as e:
        raise IndexError(str(e))
    except Exception as e:
        raise Exception(f"Erreur lors de la lecture du PDF: {str(e)}")



In [4]:
reader = PdfReader("CV_V4_EN.pdf")
page = reader.pages[0] # 1 seule page
text_brut = page.extract_text()
print(text_brut)
print(type(text_brut))
print(len(text_brut))

Quentin Lim  
Last year Student in 
Data Science
quentinlim 384@ gm ail.com
+33782885515
linkedin.com /in/quentin-lim-
978746250
github.com /QL2111
Profile
Currently seeking an internship in 
Data Science starting in March 
2025 for a duration of 4 to 6 
months.
Skills
M L, NLP, TensorFlow; Scikit-learn, 
M LFlow, Transform ers, LLM , Tim e 
Series.
SQL, NoSQL, M ongoDB
Qlik, Tableau, PowerBI
OpenCV, YOLO
Azure, AW S, Fabric
Git/Agile m ethodology
Python/R/Java/PySpark/JavaScript/
PHP
Gen AI
Interests
Board Games: Casual boarding
gam es and Chess club
Gaming: Played in competitives
tournam ents, peaked GrandMaster
in League of Legends
Sailing: M em ber of the sailing club
at La Rochelle Université
Basket ball: Sports association
during M iddle School, teamwork
and fast decision making.
Volunteering
ESN Cosmolyon
2023 – present
Vice-President, holdings Culturals 
Event for internationals and local 
student(Board gam es/Day 
trips/Spanish dinner)
Alter-ego Lyon 2
2023 – present
Buddy Sys

In [5]:
# print(read_pdf("CV_V4_EN.pdf"))

### Model Mistral Mini -> Reformulation Texte

In [6]:
# TODO
# Formulation du prompt (CV texte brut + )
# Ajouter les suivis de prix + impact ecologique ?


In [7]:
import os
import litellm
import json
from dotenv import load_dotenv, find_dotenv


load_dotenv()



True

In [8]:
reformulation_prompt = f"""
A partir d'un texte brut venant d'un CV, extrait les informations suivantes au format JSON :
{{
    "Diplome": [
      {{
          "niveau_etudes": "int",
          "domaine_etudes": ["str"]
      }}
    ],

    "Competences": ["str"],
    "Experiences": [
        {{
            "domaine_activite": ["str"],
            "poste_occupe": "str",
            "duree": "int"
        }}
    ],
    "Profil": {{
        "titre": "str",
        "disponibilite": "str"
    }}
}}

Texte du CV :
"{text_brut}"

Ne retourne que le JSON
"""
reformulation_prompt

'\nA partir d\'un texte brut venant d\'un CV, extrait les informations suivantes au format JSON :\n{\n    "Diplome": [\n      {\n          "niveau_etudes": "int",\n          "domaine_etudes": ["str"]\n      }\n    ],\n\n    "Competences": ["str"],\n    "Experiences": [\n        {\n            "domaine_activite": ["str"],\n            "poste_occupe": "str",\n            "duree": "int"\n        }\n    ],\n    "Profil": {\n        "titre": "str",\n        "disponibilite": "str"\n    }\n}\n\nTexte du CV :\n"Quentin Lim  \nLast year Student in \nData Science\nquentinlim 384@ gm ail.com\n+33782885515\nlinkedin.com /in/quentin-lim-\n978746250\ngithub.com /QL2111\nProfile\nCurrently seeking an internship in \nData Science starting in March \n2025 for a duration of 4 to 6 \nmonths.\nSkills\nM L, NLP, TensorFlow; Scikit-learn, \nM LFlow, Transform ers, LLM , Tim e \nSeries.\nSQL, NoSQL, M ongoDB\nQlik, Tableau, PowerBI\nOpenCV, YOLO\nAzure, AW S, Fabric\nGit/Agile m ethodology\nPython/R/Java/PyS

In [9]:
# Vérification de la clé API
#  api_key=os.getenv("MISTRAL_API_KEY")
# api_key

In [10]:
# litellm._turn_on_debug()
# Température, plus c'est proche de 1, plus c'est créatif
# On cherche à savoir combien de token on devrait mettre

CV_reformuler = litellm.completion(
            model="mistral/ministral-3b-latest",  # Choix d'un modèle petit
            messages=[{"role": "user", "content": reformulation_prompt}],
            max_tokens=1500,
            temperature=0.3,
            api_key=os.getenv("MISTRAL_API_KEY"),
        )

resultat = CV_reformuler["choices"][0]["message"][
            "content"
        ].strip()

In [11]:
# Le CV fait 4231 en len
# len de 1863 pour 500 max tokens
# len de 4429 pour 1500 max tokens -> C'est OK, on essaye avec 1000 max tokens
print(resultat)
print(len(resultat))



```json
{
    "Diplome": [
        {
            "niveau_etudes": "Master",
            "domaine_etudes": ["Data Science"]
        }
    ],
    "Competences": [
        "M L, NLP, TensorFlow; Scikit-learn, M LFlow, Transform ers, LLM , Tim e Series.",
        "SQL, NoSQL, M ongoDB",
        "Qlik, Tableau, PowerBI",
        "OpenCV, YOLO",
        "Azure, AW S, Fabric",
        "Git/Agile m ethodology",
        "Python/R/Java/PySpark/JavaScript/",
        "PHP",
        "Gen AI"
    ],
    "Experiences": [
        {
            "domaine_activite": ["Data Science"],
            "poste_occupe": "Student Job at the IT department of the university",
            "duree": 1
        },
        {
            "domaine_activite": ["Data Science"],
            "poste_occupe": "Internship Data Analyst",
            "duree": 1
        }
    ],
    "Profil": {
        "titre": "Currently seeking an internship in Data Science starting in March 2025 for a duration of 4 to 6 months.",
        "disponib

In [12]:
def analyze_cv(text_brut: str, temperature: float = 0.01, max_tokens: int = 1500) -> str:
    """
    Analyse un CV en format texte et retourne les informations structurées en JSON.

    Args:
        text_brut (str): Texte brut du CV à analyser
        temperature (float, optional): Paramètre de créativité du modèle. Defaults to 0.2.
        max_tokens (int, optional): Nombre maximum de tokens pour la réponse. Defaults to 1500.

    Returns:
        str: Informations en string(réponse du LLM) structurées du CV au format JSON

    Raises:
        Exception: En cas d'erreur d'API ou de parsing JSON
    """
    try:
        reformulation_prompt = f"""
        À partir du texte brut d'un CV, extrais les informations suivantes au format JSON. 
        Si une information n'est pas explicitement mentionnée dans le texte, retourne `null` ou une liste vide.
        Ne devine pas les informations manquantes et ne retourne que ce qui est clairement présent dans le texte.

        Format JSON attendu :
        {{
            "Formation": [
                {{
                    "niveau_etudes": "str",  // Ex: "Licence", "Master", "Doctorat"
                    "domaine_etudes": ["str"]  // Ex: ["Informatique", "Mathématiques"]
                }}
            ],
            "Competences": ["str"],  // Ex: ["Python", "Gestion de projet"]
            "Experiences": [
                {{
                    "domaine_activite": ["str"],  // Ex: ["Tech", "Finance"]
                    "poste_occupe": "str",  // Ex: "Développeur Python"
                    "duree": "str"  // Ex: "2 ans"
                }}
            ],
            "Profil": {{
                "titre": "str",  // Ex: "Développeur Full-Stack"
                "disponibilite": "str"  // Ex: "dd-mm-yyyy"
            }}
        }}

        Texte du CV :
        "{text_brut}"

        Ne retourne que le JSON, sans commentaires supplémentaires.
        """

        CV_reformuler = litellm.completion(
            model="mistral/mistral-medium",
            messages=[{"role": "user", "content": reformulation_prompt}],
            max_tokens=max_tokens,
            temperature=temperature,
            api_key=os.getenv("MISTRAL_API_KEY"),
        )

        # Extraire et parser le JSON de la réponse
        resultat = CV_reformuler["choices"][0]["message"]["content"].strip()
        return resultat
    
    except Exception as e:
        raise Exception(f"Erreur lors de l'analyse du CV: {str(e)}")

In [13]:
CV_reformuler = analyze_cv(text_brut)
print(CV_reformuler)

{
  "Formation": [
    {
      "niveau_etudes": "Master",
      "domaine_etudes": ["Data Science"]
    }
  ],
  "Competences": [
    "ML",
    "NLP",
    "TensorFlow",
    "Scikit-learn",
    "MLFlow",
    "Transformers",
    "LLM",
    "Time Series",
    "SQL",
    "NoSQL",
    "MongoDB",
    "Qlik",
    "Tableau",
    "PowerBI",
    "OpenCV",
    "YOLO",
    "Azure",
    "AWS",
    "Fabric",
    "Git/Agile methodology",
    "Python",
    "R",
    "Java",
    "PySpark",
    "JavaScript",
    "PHP",
    "Gen AI"
  ],
  "Experiences": [
    {
      "domaine_activite": [],
      "poste_occupe": "Vice-President, holdings Culturals Events for internationals and local students",
      "duree": "2023 – present"
    },
    {
      "domaine_activite": [],
      "poste_occupe": "Buddy System with Lyon 2 University",
      "duree": "2023 – present"
    },
    {
      "domaine_activite": ["Data Science"],
      "poste_occupe": "Student Job at the IT department of the university, help desk, loan o

#### Matching CV et offre :

- Obtention du string job offer par le scrapping
- Transformation via un LLM de l'offre à un format similaire au CV_reformuler
- Embedding + calcul de la similarité cosinus globale
- Transformation en JSON du CV_reformuler et job_offer_reforumuler pour pouvoir effectuer une similarité par partie(clef communes car passé tout les deux par un LLM)
- Calcul des similarité par parties

In [14]:



# Example job offer
job_offer = """
  Titre : Data scientist (H/F) SALON TAF Toulouse 2025
  Entreprise : AOSIS CONSULTING
  Contrat : CDI
  Localisation : 31 - TOULOUSE
  Description :
  Concrètement qu'est-ce qu'on fait ?
  Nous proposons des Services, des Logiciels et de la Formation dans le domaine de la Business Intelligence, de la Data Science, de la Gouvernance et de la Visualisation de la donnée. L'Infrastructure et le Développement font également partie du panel de nos compétences.
  Notre leitmotiv ? Citoyenneté, écologie et sport avec des experts de la donnée.
  Notre objectif ? Porter encore plus haut et plus fort nos valeurs de sport, d'écologie et de citoyenneté et accompagner nos consultants dans leur montée en compétence.

  Le poste
  Dans le cadre d'une ouverture de poste, nous sommes à la recherche d'un.e Data Scientist pour l'un de nos clients.
  Un.e data scientist senior (5 ans d'expérience) très autonome dans ses analyses et explorations, expérimenté dans l'industrialisation des modèles dans des environnements cloud.

  Vous interviendrez sur toutes les étapes du projet, vous devrez :

  Identifier leurs difficultés et proposer des solutions
  Evaluer la qualité des données, les nettoyer, les agréger, et mener des études adhoc
  Concevoir, implémenter et comparer différents algorithmes et méthodes statistiques
  Concevoir et mener des protocoles de test en conditions réelles (magasins, entrepôts, .)
  Développer des outils de visualisation des données
  Mettre en production, maintenir et itérer sur les solutions

  Environnement technique: Python, SQL, Terraform, Bitbucket, Jenkins, Airflow, Docker, Kubernetes, GCP (BigQuery, Spark DataProc)

  Profil recherché
  Vous êtes issu.e d'un Master en informatique ou mathématiques appliquées.
  Vous avez à minima 5 années d'expériences en Data Science et maitrisez les algorithmes d'apprentissage statistique.
  Vous maîtrisez Python et SQL
  Vous avez une expérience dans le développement logiciel agile avec multiples contributeurs
  Vous avez une expérience du cloud, idéalement GCP

  Pourquoi nous rejoindre ?
  Parce que nous sommes une société à taille humaine
  Parce que nous nous engageons (réellement) dans des enjeux importants : l'écologie, la citoyenneté et le sport
  Mais aussi parce que nous aimons réaliser tout un tas d'activités : des jeux, des afterworks, des restaurants, ... et plein d'autres choses encore :)
"""


#### Reformulation de l'offre d'emploi :

In [15]:
#### Transformation de l'offre d'emploi au même format que le CV
def analyze_offre_emploi(offre: str, temperature: float = 0.01, max_tokens: int = 1500) -> str:
    """
    Analyse une offre d'emploi en format texte et retourne les informations structurées en JSON.

    Args:
        text_brut (str): Texte brut del'offre à analyser
        temperature (float, optional): Paramètre de créativité du modèle. Defaults to 0.2.
        max_tokens (int, optional): Nombre maximum de tokens pour la réponse. Defaults to 1500.

    Returns:
        str: Informations en string(réponse du LLM) structurées du CV au format JSON

    Raises:
        Exception: En cas d'erreur d'API ou de parsing JSON
    """
    try:
        reformulation_prompt = f"""
        À partir de cette offre d'emploi, extrais les informations suivantes au format JSON. 
        Si une information n'est pas explicitement mentionnée dans le texte, retourne `null` ou une liste vide.
        De plus la disponibilité correspond à la date de début du poste.
        Ne devine pas les informations manquantes et ne retourne que ce qui est clairement présent dans le texte.
        
        Format JSON attendu :
        {{
            "Formation": [
                {{
                    "niveau_etudes": "str",  // Ex: "Licence", "Master", "Doctorat"
                    "domaine_etudes": ["str"]  // Ex: ["Informatique", "Mathématiques"]
                }}
            ],
            "Competences": ["str"],  // Ex: ["Python", "Gestion de projet"]
            "Experiences": [
                {{
                    "domaine_activite": ["str"],  // Ex: ["Tech", "Finance"]
                    "poste_occupe": "str",  // Ex: "Développeur Python"
                    "duree": "str"  // Ex: "2 ans"
                }}
            ],
            "Profil": {{
                "titre": "str",  // Ex: "Développeur Full-Stack"
                "disponibilite": "str"  // Ex: "dd-mm-yyyy"
            }}
        }}

        Texte de l'offre d'emploi :
        "{offre}"

        Ne retourne que le JSON, sans commentaires supplémentaires.
        """

        offre_reformuler = litellm.completion(
            model="mistral/mistral-medium",
            messages=[{"role": "user", "content": reformulation_prompt}],
            max_tokens=max_tokens,
            temperature=temperature,
            api_key=os.getenv("MISTRAL_API_KEY"),
        )

        # Extraire et parser le JSON de la réponse
        resultat = offre_reformuler["choices"][0]["message"]["content"].strip()
        return resultat
    
    except Exception as e:
        raise Exception(f"Erreur lors de l'analyse du CV: {str(e)}")

In [16]:
job_offer_reforumer = analyze_offre_emploi(job_offer)
print(job_offer_reforumer)

{
  "Formation": [
    {
      "niveau_etudes": "Master",
      "domaine_etudes": ["Informatique", "Mathématiques appliquées"]
    }
  ],
  "Competences": [
    "Python",
    "SQL",
    "Terraform",
    "Bitbucket",
    "Jenkins",
    "Airflow",
    "Docker",
    "Kubernetes",
    "GCP (BigQuery, Spark DataProc)",
    "Développement logiciel agile",
    "Algorithmes d'apprentissage statistique"
  ],
  "Experiences": [
    {
      "domaine_activite": ["Data Science"],
      "poste_occupe": "Data Scientist",
      "duree": "5 ans"
    }
  ],
  "Profil": {
    "titre": "Data Scientist",
    "disponibilite": null
  }
}


#### Similarité cosinus globale


In [17]:
# print(HF_TOKEN)

In [18]:
import requests
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Token d'API Hugging Face
HF_TOKEN = os.getenv("HF_TOKEN")
API_URL = "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"


def get_embedding(text: str) -> np.ndarray:
    """
    Envoie une requête à l'API Hugging Face pour obtenir l'embedding d'un texte.

    Args:
        text (str): Texte à encoder.

    Returns:
        np.ndarray: Embedding du texte.
    """
    headers = {"Authorization": f"Bearer {os.getenv("HF_TOKEN")}"}
    response = requests.post(
        "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        headers=headers,
        json={"inputs": text, "options": {"wait_for_model": True}},
    )
    if response.status_code != 200:
        raise Exception(f"Erreur API : {response.status_code}, {response.text}")
    return np.array(response.json())

In [19]:
import numpy as np

def calculate_similarity(text1: str, text2: str) -> float:
    """
    Calcule la similarité cosinus entre deux textes en utilisant l'API Hugging Face.
    
    Args:
        text1 (str): Premier texte.
        text2 (str): Deuxième texte.
    
    Returns:
        float: Score de similarité cosinus entre les deux textes.
    """
    # Obtenir les embeddings des textes
    embedding1 = get_embedding(text1)
    embedding2 = get_embedding(text2)
    
    # Calculer la similarité cosinus
    similarity = cosine_similarity([embedding1], [embedding2])[0][0]
    return similarity

# Calculer la similarité entre le CV et l'offre d'emploi
similarity_score = calculate_similarity(CV_reformuler, job_offer_reforumer)
print(f"Similarité globale entre le CV et l'offre d'emploi: {similarity_score:.4f}")


Similarité globale entre le CV et l'offre d'emploi: 0.9063


#### Conversion en JSON-like

In [20]:

# Calcul de la similarité entre chaque partie du CV et l'offre d'emploi
# Diviser le CV reformulé en json en plusieurs parties
cv_json = json.loads(CV_reformuler)
# print(cv_json)
# convert job_offer to json
job_offer_json = json.loads(job_offer_reforumer)

In [21]:
print(job_offer_json)

{'Formation': [{'niveau_etudes': 'Master', 'domaine_etudes': ['Informatique', 'Mathématiques appliquées']}], 'Competences': ['Python', 'SQL', 'Terraform', 'Bitbucket', 'Jenkins', 'Airflow', 'Docker', 'Kubernetes', 'GCP (BigQuery, Spark DataProc)', 'Développement logiciel agile', "Algorithmes d'apprentissage statistique"], 'Experiences': [{'domaine_activite': ['Data Science'], 'poste_occupe': 'Data Scientist', 'duree': '5 ans'}], 'Profil': {'titre': 'Data Scientist', 'disponibilite': None}}


In [22]:
print(type(cv_json)) # Les données sont en string JSON like
print(type(job_offer_json)) # Les données sont en string JSON like

<class 'dict'>
<class 'dict'>


#### Calcul des similarités cosinus par parties :

- Formation
- Competences
- Experiences
- Profil

In [23]:
# Similarité entre les deux json like

# Similarité Formation
# print(cv_json.get("Formation"))
# print(job_offer_json.get("Formation"))


In [24]:
# Similarité entre Formation attendu et Formation du CV
formation_candidat = cv_json.get("Formation")
formation_attendu = job_offer_json.get("Formation")

similarite_formation = calculate_similarity(str(formation_candidat), str(formation_attendu))
print(f"Similarité entre Formation attendu et Formation du CV: {similarite_formation:.4f}")


# Similarité entre Compétences attendu et Compétences du CV
competences_candidat = cv_json.get("Competences")
# print(competences_candidat)
competences_attendu = job_offer_json.get("Competences")
# print(competences_attendu)

similarite_competences = calculate_similarity(str(competences_candidat), str(competences_attendu))
print(f"Similarité entre Compétences attendu et Compétences du CV: {similarite_competences:.4f}")


# Similarité entre Experiences attendu et Experiences du CV
experiences_candidat = cv_json.get("Experiences")
# print(experiences_candidat)
experiences_attendu = job_offer_json.get("Experiences")
# print(experiences_attendu)

similarite_experiences = calculate_similarity(str(experiences_candidat), str(experiences_attendu))
print(f"Similarité entre Experiences attendu et Experiences du CV: {similarite_experiences:.4f}")


# Similarité entre Profil attendu et Profil du CV
profil_candidat = cv_json.get("Profil")
print(profil_candidat)
profil_attendu = job_offer_json.get("Profil")
print(profil_attendu)

similarite_profil = calculate_similarity(str(profil_candidat), str(profil_attendu))
print(f"Similarité entre Profil attendu et Profil du CV: {similarite_profil:.4f}")

###################### Voir si on ne peut pas faire mieux (check if any au lieu d'une simalarité globale) ######################
### Au lieu d'installer sentence transformers, utiliser une API huggingface avec pipeline et requests


Similarité entre Formation attendu et Formation du CV: 0.7965
Similarité entre Compétences attendu et Compétences du CV: 0.8096
Similarité entre Experiences attendu et Experiences du CV: 0.5714
{'titre': 'Currently seeking an internship in Data Science starting in March 2025 for a duration of 4 to 6 months.', 'disponibilite': 'March 2025'}
{'titre': 'Data Scientist', 'disponibilite': None}
Similarité entre Profil attendu et Profil du CV: 0.5046


In [25]:
def calculate_section_similarity(cv_reformule: str, offre_emploi_reformule: str) -> dict:
    """
    Calcule les similarités détaillées entre les différentes sections d'un CV et une offre d'emploi.
    
    Args:
        cv_reformule (str): Informations du CV au format JSON string
        offre_emploi_reformule (str): Informations de l'offre d'emploi au format JSON string
        
    Returns:
        dict: Dictionnaire contenant les scores de similarité pour chaque section
        
    Exemple:
        {
            'formation': float,
            'competences': float,
            'experiences': float,
            'profil': float,
        }
    """
    try:
        # Conversion des chaînes JSON en dictionnaires
        cv_json = json.loads(cv_reformule)
        offre_emploi_json = json.loads(offre_emploi_reformule)
        
        # Calcul des similarités pour chaque section
        similarites = {}
        
        # Similarité de formation
        formation_candidat = cv_json.get("Formation", "")
        formation_attendue = offre_emploi_json.get("Formation", "")
        similarites['formation'] = calculate_similarity(
            str(formation_candidat), 
            str(formation_attendue)
        )
        
        # Similarité des compétences
        competences_candidat = cv_json.get("Competences", [])
        competences_attendues = offre_emploi_json.get("Competences", [])
        similarites['competences'] = calculate_similarity(
            str(competences_candidat), 
            str(competences_attendues)
        )
        
        # Similarité des expériences
        experiences_candidat = cv_json.get("Experiences", [])
        experiences_attendues = offre_emploi_json.get("Experiences", [])
        similarites['experiences'] = calculate_similarity(
            str(experiences_candidat), 
            str(experiences_attendues)
        )
        
        # Similarité du profil
        profil_candidat = cv_json.get("Profil", {})
        profil_attendu = offre_emploi_json.get("Profil", {})
        similarites['profil'] = calculate_similarity(
            str(profil_candidat), 
            str(profil_attendu)
        )
        
        return similarites
        
    except json.JSONDecodeError as e:
        raise ValueError(f"Erreur lors du parsing JSON : {str(e)}")
    except Exception as e:
        raise Exception(f"Erreur lors du calcul des similarités : {str(e)}")

In [26]:
calculate_section_similarity(CV_reformuler, job_offer_reforumer)

print(calculate_section_similarity(CV_reformuler, job_offer_reforumer)['formation'])

0.7964798257238572


#### Création d'une lettre de motivation à partir du CV et de l'offre

- Utilisation du CV brut car on a perdu des informations qu'on peut mettre en avant ici (bénévélot etc..)
- Même chose pour l'offre

In [27]:
# LLM pour la lettre de motivation
print(text_brut)
# job_offer




Quentin Lim  
Last year Student in 
Data Science
quentinlim 384@ gm ail.com
+33782885515
linkedin.com /in/quentin-lim-
978746250
github.com /QL2111
Profile
Currently seeking an internship in 
Data Science starting in March 
2025 for a duration of 4 to 6 
months.
Skills
M L, NLP, TensorFlow; Scikit-learn, 
M LFlow, Transform ers, LLM , Tim e 
Series.
SQL, NoSQL, M ongoDB
Qlik, Tableau, PowerBI
OpenCV, YOLO
Azure, AW S, Fabric
Git/Agile m ethodology
Python/R/Java/PySpark/JavaScript/
PHP
Gen AI
Interests
Board Games: Casual boarding
gam es and Chess club
Gaming: Played in competitives
tournam ents, peaked GrandMaster
in League of Legends
Sailing: M em ber of the sailing club
at La Rochelle Université
Basket ball: Sports association
during M iddle School, teamwork
and fast decision making.
Volunteering
ESN Cosmolyon
2023 – present
Vice-President, holdings Culturals 
Event for internationals and local 
student(Board gam es/Day 
trips/Spanish dinner)
Alter-ego Lyon 2
2023 – present
Buddy Sys

#### Extractions info perso avec un LLM

In [28]:
prompt_extraction_info_perso = f"""
    Extrait les informations personnelles suivantes du texte brut d'un CV et retourne-les au format JSON :
    - Nom et prénom
    - Email
    - Numéro de téléphone
    - Adresse

    Si une information est manquante, retourne `Non réseigner` pour cette clé.

    Texte brut du CV :
    "{text_brut}"

    Format JSON attendu :
    {{
        "nom_prenom": "str",
        "email": "str",
        "telephone": "str",
        "adresse": "str"
    }}

    Ne retourne que le JSON, sans commentaires supplémentaires.
    """



In [32]:

resultat_extraction_info_perso = litellm.completion(
            model="mistral/mistral-tiny",  
            messages=[{"role": "user", "content": prompt_extraction_info_perso}],
            max_tokens=50,
            temperature=0.1,
            api_key=os.getenv("MISTRAL_API_KEY"),
        )

resultat_extraction_info_perso_content = resultat_extraction_info_perso["choices"][0]["message"]["content"].strip()



In [ ]:
def extraction_info_perso(text_brut: str) -> str:
    """
    Extrait les informations personnelles d'un CV en format texte brut.

    Args:
        text_brut (str): Texte brut du CV à analyser

    Returns:
        str: Informations personnelles extraites du CV en format JSON like
    """

    prompt_extraction_info_perso = f"""
    Extrait les informations personnelles suivantes du texte brut d'un CV et retourne-les au format JSON :
    - Nom et prénom
    - Email
    - Numéro de téléphone
    - Adresse

    Si une information est manquante, retourne `Non réseigner` pour cette clé.

    Texte brut du CV :
    "{text_brut}"

    Format JSON attendu :
    {{
        "nom_prenom": "str",
        "email": "str",
        "telephone": "str",
        "adresse": "str"
    }}

    Ne retourne que le JSON, sans commentaires supplémentaires.
    """
    resultat_extraction_info_perso = litellm.completion(
                model="mistral/mistral-tiny",  
                messages=[{"role": "user", "content": prompt_extraction_info_perso}],
                max_tokens=50,
                temperature=0.1,
                api_key=os.getenv("MISTRAL_API_KEY"),
            )
    
    return resultat_extraction_info_perso["choices"][0]["message"]["content"].strip()
    
    

In [35]:
extraction_info_perso(text_brut)

'{\n    "nom_prenom": "Quentin Lim",\n    "email": "quentinlim384@gm ail.com",\n    "telephone": "+33782885515",\n    "adresse": "Non réseigner"\n}'

In [34]:
print(type(resultat_extraction_info_perso_content))
# convertir en json like
resultat_extraction_info_perso_json = json.loads(resultat_extraction_info_perso_content)
print(resultat_extraction_info_perso_json)

nom = resultat_extraction_info_perso_json.get("nom_prenom")
print(nom)
email = resultat_extraction_info_perso_json.get("email", "").strip()
# Remove all spaces from email
email = "".join(email.split())
print(email)  # Will print: quentinlim384@gmail.com
telephone = resultat_extraction_info_perso_json.get("telephone")
print(telephone)
adresse = resultat_extraction_info_perso_json.get("adresse")
print(adresse)


# Set French locale
try:
    locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')  # Linux/Mac
except:
    locale.setlocale(locale.LC_TIME, 'fra_fra')  # Windows

# Get date in French format
date = datetime.now().strftime("%d %B %Y")
print(date)  # Output: 13 mars 2024

<class 'str'>


JSONDecodeError: Unterminated string starting at: line 4 column 18 (char 93)

In [ ]:
prompt_lettre_motivation = f"""
Tu es un assistant qui rédige des lettres de motivation personnalisées et professionnelles. Voici les informations nécessaires :

**Informations du candidat :**
- Nom : {nom}
- Email : {email}
- Téléphone : {telephone}
- Adresse : {adresse}
- Date : {date}

**Texte brut du CV :**
{text_brut}

**Offre d'emploi :**
{job_offer}

**Instructions :**
1. Rédige une lettre de motivation au format A4, bien structurée et professionnelle.
2. Mets en avant les compétences et expériences du candidat qui correspondent aux exigences du poste.
3. Mentionne des éléments spécifiques de l'entreprise ou du poste pour montrer que la candidature est personnalisée.
4. Explique pourquoi le candidat est motivé pour rejoindre cette entreprise en particulier.
5. Utilise un ton professionnel et évite les phrases génériques.
6. Ne laisse pas de champs vides et ne devine pas les informations manquantes.

**Format de la lettre :**
- En-tête : Nom, prénom, adresse, email, téléphone, date.
- Introduction : Présentation du candidat et motivation pour le poste.
- Corps : Compétences et expériences en lien avec le poste.
- Conclusion : Expression de l'enthousiasme et disponibilité pour un entretien.

**Exemple de structure :**
{nom}
{email}
{telephone}
{adresse}
{date}

Madame, Monsieur,
Je me permets de vous adresser ma candidature pour le poste de [poste] au sein de [entreprise]. [Motivation personnalisée].

Avec mon expérience en [domaine] et mes compétences en [compétences], je suis convaincu de pouvoir contribuer à [objectif de l'entreprise]. [Détail des expériences et compétences pertinentes].

Je serais ravi de discuter de ma candidature lors d'un entretien. Je reste à votre disposition pour toute information complémentaire.

Veuillez agréer, Madame, Monsieur, mes salutations distinguées.
{nom}
"""

In [ ]:

Lettre_motiv_genere = litellm.completion(
            model="mistral/ministral-3b-latest",  
            messages=[{"role": "user", "content": prompt_lettre_motivation}],
            max_tokens=1500,
            temperature=0.3,
            api_key=os.getenv("MISTRAL_API_KEY"),
        )

resultat = Lettre_motiv_genere["choices"][0]["message"][
            "content"
        ].strip()

In [ ]:
print(resultat)

**Quentin Lim**
12 rue de la Paix, 75000 Paris
quentinlim384@gmail.com | +33782885515
13 mars 2025

Madame, Monsieur,

Je me permets de vous adresser ma candidature pour le poste de Data Scientist au sein de AOSIS CONSULTING. Fort de mon parcours académique et professionnel en Data Science, je suis convaincu de pouvoir apporter une contribution significative à votre équipe.

Actuellement en master en Data Science à l'Université Lyon 2 Lumière, j'ai acquis une expertise avancée en analyse de données, en machine learning et en développement de modèles. Mon expérience inclut la réalisation de projets complexes tels que l'analyse de données biologiques, la création de packages R pour la régression logistique, et la mise en œuvre de solutions de visualisation des données avec Power BI et Tableau.

Au cours de mon stage en tant que Data Analyst chez SYSBLOCK, j'ai développé des compétences en dashboarding et en web development, ainsi qu'une expérience dans l'utilisation de technologies cloud

In [ ]:
def generate_lettre_motivation(text_brut: str, job_offer:str) ->str:
    """
    Génère une lettre de motivation personnalisée en fonction des informations extraites du CV et de l'offre d'emploi.
    
    Args:
        text_brut (str): Texte brut extrait du CV du candidat
        job_offer (str): Description de l'offre d'emploi
        
    Returns:
        str: Lettre de motivation générée au format texte
    
    """
    information_perso = extraction_info_perso(text_brut)
    resultat_extraction_info_perso_json = json.loads(information_perso)

    nom = resultat_extraction_info_perso_json.get("nom_prenom")
    email = resultat_extraction_info_perso_json.get("email", "").strip()
    # Remove all spaces from email
    email = "".join(email.split())
    telephone = resultat_extraction_info_perso_json.get("telephone")
    adresse = resultat_extraction_info_perso_json.get("adresse")


    # Set French locale
    try:
        locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')  # Linux/Mac
    except:
        locale.setlocale(locale.LC_TIME, 'fra_fra')  # Windows

    # Get date in French format
    date = datetime.now().strftime("%d %B %Y")

    prompt_lettre_motivation = f"""
    Tu es un assistant qui rédige des lettres de motivation personnalisées et professionnelles. Voici les informations nécessaires :

    **Informations du candidat :**
    - Nom : {nom}
    - Email : {email}
    - Téléphone : {telephone}
    - Adresse : {adresse}
    - Date : {date}

    **Texte brut du CV :**
    {text_brut}

    **Offre d'emploi :**
    {job_offer}

    **Instructions :**
    1. Rédige une lettre de motivation au format A4, bien structurée et professionnelle.
    2. Mets en avant les compétences et expériences du candidat qui correspondent aux exigences du poste.
    3. Mentionne des éléments spécifiques de l'entreprise ou du poste pour montrer que la candidature est personnalisée.
    4. Explique pourquoi le candidat est motivé pour rejoindre cette entreprise en particulier.
    5. Utilise un ton professionnel et évite les phrases génériques.
    6. Ne laisse pas de champs vides et ne devine pas les informations manquantes.

    **Format de la lettre :**
    - En-tête : Nom, prénom, adresse, email, téléphone, date.
    - Introduction : Présentation du candidat et motivation pour le poste.
    - Corps : Compétences et expériences en lien avec le poste.
    - Conclusion : Expression de l'enthousiasme et disponibilité pour un entretien.

    **Exemple de structure :**
    {nom}
    {email}
    {telephone}
    {adresse}
    {date}

    Madame, Monsieur,
    Je me permets de vous adresser ma candidature pour le poste de [poste] au sein de [entreprise]. [Motivation personnalisée].

    Avec mon expérience en [domaine] et mes compétences en [compétences], je suis convaincu de pouvoir contribuer à [objectif de l'entreprise]. [Détail des expériences et compétences pertinentes].

    Je serais ravi de discuter de ma candidature lors d'un entretien. Je reste à votre disposition pour toute information complémentaire.

    Veuillez agréer, Madame, Monsieur, mes salutations distinguées.
    {nom}
    """
        
    Lettre_motiv_genere = litellm.completion(
                model="mistral/ministral-3b-latest",  
                messages=[{"role": "user", "content": prompt_lettre_motivation}],
                max_tokens=1500,
                temperature=0.3,
                api_key=os.getenv("MISTRAL_API_KEY"),
            )

    resultat = Lettre_motiv_genere["choices"][0]["message"][
                "content"
            ].strip()
    
    return resultat



In [ ]:
print(generate_lettre_motivation(text_brut, job_offer))

**Quentin Lim**
quentinlim384@gmail.com
+33782885515
Non réseigner
13 mars 2025

Madame, Monsieur,

Je me permets de vous adresser ma candidature pour le poste de Data Scientist au sein de AOSIS CONSULTING. Fort de mon parcours académique et professionnel en Data Science, je suis convaincu de pouvoir apporter une contribution significative à votre équipe.

Actuellement étudiant en Master en Data Science à l'Université Lyon 2 Lumière, j'ai acquis une expertise approfondie en analyse de données avancée, en NLP/LLM, en vision par ordinateur et en apprentissage profond. Mon expérience inclut la réalisation de projets complexes tels que la création d'un package de régression logistique en Python, l'analyse de sentiments sur les avis TripAdvisor, et la gestion de projets cloud sur AWS.

Mon parcours académique et professionnel m'a permis de développer des compétences solides en Python, SQL, et en développement logiciel agile. J'ai également une expérience significative dans le développement 

In [ ]:
"[Votre Nom]\n[Votre Adresse]\n[Code Postal, Ville]\n[Votre Email]\n[Votre Numéro de Téléphone]\n[Date]\n\nAOSIS CONSULTING\n[Adresse de l'entreprise]\n31 - Toulouse\n\nObjet : Candidature au poste de Data Scientist (H/F)\n\nMadame, Monsieur,\n\nJe me permets de vous adresser ma candidature pour le poste de Data Scientist au sein de votre entreprise, tel que publié sur votre site. Actuellement étudiant en Data Science à l'Université Lumière Lyon 2, je suis à la recherche d'une opportunité d'internship pour débuter ma carrière dans ce domaine passionnant.\n\nMon parcours académique et professionnel m'a permis de développer une solide expertise en data science, notamment en matière d'analyse avancée, de Big Data, de MLOps, et de predictive analytics. J'ai eu l'occasion de travailler sur divers projets qui m'ont permis de maîtriser des outils tels que Python, R, SQL, et des bibliothèques comme Scikit-learn, TensorFlow, et PySpark. Mon expérience inclut également la création de packages logiciels, la réalisation d'analyses NLP, et la gestion de bases de données NoSQL.\n\nAu cours de mon stage en tant que Data Analyst chez SYSBLOCK, j'ai eu l'opportunité de travailler sur des projets de dashboarding et d'analyse de données, ainsi que sur des projets de développement web et de blockchain. J'ai également développé des compétences en DevOps, notamment sur AWS, et j'ai une expérience significative avec des outils de visualisation comme Power BI et Tableau.\n\nCe qui m'attire particulièrement dans votre offre est votre engagement envers des valeurs importantes telles que la citoyenneté, l'écologie et le sport. Ces valeurs résonnent profondément avec moi et je suis convaincu que mon profil pourrait apporter une contribution significative à vos projets.\n\nJe suis particulièrement attiré par l'opportunité de travailler dans un environnement technique avancé, utilisant des outils tels que Python, SQL, Terraform, et des plateformes cloud comme GCP. Mon expérience en développement logiciel agile et ma maîtrise des méthodologies Git/Agile me permettent de m'intégrer rapidement et de collaborer efficacement avec des équipes multidisciplinaires.\n\nJe suis convaincu que mon parcours et mes compétences correspondent parfaitement aux exigences de ce poste. Je serais ravi de pouvoir discuter plus en détail de ma candidature lors d'un entretien.\n\nJe vous remercie par avance pour l'attention que vous porterez à ma demande et je reste à votre disposition pour toute information complémentaire.\n\nDans l'attente de votre réponse, je vous prie d'agréer, Madame, Monsieur, l'expression de mes salutations distinguées.\n\n[Votre Nom]"

"[Votre Nom]\n[Votre Adresse]\n[Code Postal, Ville]\n[Votre Email]\n[Votre Numéro de Téléphone]\n[Date]\n\nAOSIS CONSULTING\n[Adresse de l'entreprise]\n31 - Toulouse\n\nObjet : Candidature au poste de Data Scientist (H/F)\n\nMadame, Monsieur,\n\nJe me permets de vous adresser ma candidature pour le poste de Data Scientist au sein de votre entreprise, tel que publié sur votre site. Actuellement étudiant en Data Science à l'Université Lumière Lyon 2, je suis à la recherche d'une opportunité d'internship pour débuter ma carrière dans ce domaine passionnant.\n\nMon parcours académique et professionnel m'a permis de développer une solide expertise en data science, notamment en matière d'analyse avancée, de Big Data, de MLOps, et de predictive analytics. J'ai eu l'occasion de travailler sur divers projets qui m'ont permis de maîtriser des outils tels que Python, R, SQL, et des bibliothèques comme Scikit-learn, TensorFlow, et PySpark. Mon expérience inclut également la création de packages lo